In [2]:
#htmlを解析しデータを抽出するためのライブラリ
from bs4 import BeautifulSoup

#httpリクエストを行うためのライブラリ
#webページの情報を取得する
import requests

#すべて表示するプリント
from pprint import pprint

#問題を日数に割り当てるためのライブラリ
from datetime import datetime, timedelta

In [3]:
def get_soup(url):
    # requestsを使ってURLからHTMLコンテンツを取得
    response = requests.get(url)

    # 取得したHTMLコンテンツ
    html_content = response.text

    # BeautifulSoupを使ってHTMLを解析
    soup = BeautifulSoup(html_content, 'html.parser')

    return soup

def get_find_a(soup,url):
    #配列を宣言
    links = []
    # 特定の条件にマッチするすべてのリンクを検索し、表示
    for link in soup.find_all('a', href=True):
        href = link['href']
        # 'https://kakomonn.com/keamane/list1/'で始まるURLに限定
        if href.startswith(url):
            links.append(href)
    return links

In [4]:
#メインページのsoup
soup_main = get_soup('https://kakomonn.com/keamane')
home_links = get_find_a(soup_main,'https://kakomonn.com/keamane/list1/')

In [5]:
ques_ini_num = []
for link in home_links:
    soup_sub = get_soup(link)
    ques_links = get_find_a(soup_sub, 'https://kakomonn.com/keamane/questions/')
    url = str(ques_links[0])
    ques_ini_num.append(url[-5:])

In [12]:
# リスト内包表記を用いて文字列のリストを整数のリストに変換
ques_ini_num = [int(item) for item in ques_ini_num]

# 開始日付を定義
start_date = datetime(2024, 4, 3)

# 各問題番号とその日付を格納するリスト
dates_for_questions = []

# 各セットとその中の各問題に対して処理
for i, start_number in enumerate(ques_ini_num):
    for offset in range(60):  # 各セットは60問を含む
        question_date = start_date + timedelta(days=i*60 + offset)  # 日付を計算
        question_number = start_number + offset  # 問題番号を計算
        dates_for_questions.append((question_number, question_date.strftime('%Y/%m/%d')))

In [ ]:
for i in ques_ini_num:
    i = int(i)
    for j in range(i, i + 59):     
        url = 'https://kakomonn.com/keamane/questions/' + str(j)
        soup = get_soup(url)

        txt = ""
        # タイトルを取得
        title = soup.find("h1", class_="ctr_h1").text
        txt += title + "\n"

        # 質問文を取得
        question = soup.find("div", class_="centerbody01_24").text
        txt += question + "\n"

        # 選択肢を取得
        options = [option.text.strip() for option in soup.find_all("div", class_="centerbody01_26")]
        for i, option in enumerate(options, 1):
            txt += f"{i}. {option}" + "\n"

        txt += url + "\n"
        print(txt)